In [13]:
import torch
import sys
import yaml
from torchvision import transforms, datasets
import torchvision
import numpy as np
import os
import pickle
from sklearn import preprocessing
from torch.utils.data.dataloader import DataLoader

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
import torchmetrics

from Rosseta.resnet_base_network import ResNet18
from util.loader_1 import DataTraffic
import joblib
from Rosseta.lstm import LSTMCell,LSTMModel

input_dim = 32
seq_dim = 16
device = 'cpu'

Helper Function

In [14]:
def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []

    # get the features from the pre-trained model
    for i, (x, y) in enumerate(loader):
        with torch.no_grad():
            feature_vector = encoder(x[0])
            x_train.extend(feature_vector)
            y_train.extend(y.numpy())

    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    return x_train, y_train

def generate_encoder():
    device = 'cpu'
    config = yaml.load(open("Rosseta/model/checkpoints/config.yaml", "r"), Loader=yaml.FullLoader)
    encoder = ResNet18(**config['network'])

    load_params = torch.load(os.path.join('/home/mininet/experiment/Cross-network/Rosseta/test/Rosseta/model/checkpoints/model.pth'),
                            map_location=torch.device(torch.device(device)))

    if 'online_network_state_dict' in load_params:
        encoder.load_state_dict(load_params['online_network_state_dict'])
        print("Parameters successfully loaded.")
    encoder = torch.nn.Sequential(*list(encoder.children())[:-1])    
    encoder = encoder.to(device)
    encoder.eval()
    return encoder

def create_data_loaders_from_arrays(X_test, y_test):
    test = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=128, shuffle=True)
    return test_loader

def generate_testFeature(file_name, encoder):
    test_dataset = DataTraffic(file_name, dataset = 'test')
    test_loader = DataLoader(test_dataset, batch_size=128,
                          num_workers=0, drop_last=False, shuffle=False)

    x_test, y_test = get_features_from_encoder(encoder, test_loader)
    x_test = torch.mean(x_test, dim=[2, 3])
    scaler = pickle.load(open('util/sc.pkl','rb'))
    x_test = scaler.transform(x_test).astype(np.float32)
    x_test = np.pad(x_test, ((0, 0), (0, 512 - len(x_test[0]))), 'constant').reshape(-1, 1, 32, 16)
    test = torch.utils.data.TensorDataset(torch.from_numpy(x_test), y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=128, shuffle=False)
    return test_loader

def runmodel(test_loader, model):
    accuracy = torchmetrics.Accuracy()
    recall = torchmetrics.Recall(average='macro', num_classes=10)
    precision = torchmetrics.Precision(average='macro', num_classes=10)
    f1score = torchmetrics.F1(average='macro', num_classes=10)
    eval_loss = 0
    eval_acc = 0

    for images, labels in test_loader:
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
        else:
            images = Variable(images.view(-1 , seq_dim, input_dim))
        
        # Forward pass only to get logits/output
        outputs = model(images)
        
        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)
        
        
        labels = labels.cpu()
        predicted = predicted.cpu()
        accuracy(predicted, labels)
        recall(predicted, labels)
        precision(predicted, labels)
        f1score(predicted, labels)


    acc = accuracy.compute().data.detach()
    print('Accuracy:',acc)
    rec = recall.compute().data.detach()
    prec = precision.compute().data.detach()
    f1sc = f1score.compute().data.detach()
    print(f" Recall: {rec}, precision: {prec}, f1score: {f1sc}")

testFiles

In [15]:

dir = 'data/'
testFile = dir + 'test-cn-bin.csv'
testFile1 = dir + 'test-kr-bin.csv'
testFile2 = dir + 'test-us-bin.csv'
testFile3 = dir + 'test-wifi-bin.csv'
testFile4 = dir + 'test-4G-bin.csv'
testFile5 = dir + 'test-3G-bin.csv'

encoder = generate_encoder()
test_loader = generate_testFeature(testFile5, encoder)
model = torch.load('Rosseta/lstm-Rosseta.pth')
runmodel(test_loader, model)


Parameters successfully loaded.


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Accuracy: tensor(0.9476)
 Recall: 0.9519410133361816, precision: 0.9388858079910278, f1score: 0.9444727897644043
